In [ ]:
import sqlite3  # For SQLite
import psycopg2  # For PostgreSQL
import pymysql  # For MySQL
# import pyodbc  # For Microsoft SQL Server

import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import seaborn as sns

In [ ]:
conn = sqlite3.connect('mydatabase.db')  # Connect to an SQLite database file or create one
cursor = conn.cursor() #need the cursor to be able to execute SQL commands in Python

In [ ]:
#what catherine had

def create_sqlite_db_from_csv(csv_url, db_name, table_name):
    conn = sqlite3.connect(db_name)
    with open(csv_url, "r") as f:
        data_df = pd.read_csv(f)

    data_df.to_sql(table_name, conn, if_exists="replace", index=False)
    conn.commit()
    conn.close()
    print(f"Database {db_name} with table {table_name} created successfully!")

In [ ]:
# # add datasets to database, general format 
# datasets = ["Amazon Sale Report.csv"] #add the the csv files in string form here

# for table in datasets:
#     create_sqlite_db_from_csv(table, "mydatabase.db", table[:-4]) #add each table into the database with .csv in name for table

In [ ]:
create_sqlite_db_from_csv('Amazon Sale Report.csv', 'mydatabase.db', 'amzn')

/Users/angus/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/angus/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


Database mydatabase.db with table amzn created successfully!


In [ ]:
conn = sqlite3.connect('mydatabase.db')  # Connect to an SQLite database file or create one
cursor = conn.cursor() #need the cursor to be able to execute SQL commands in Python

cursor.execute("SELECT * FROM amzn")
results = cursor.fetchall()
results

[(0,
  '405-8078784-5731545',
  '04-30-22',
  'Cancelled',
  'Merchant',
  'Amazon.in',
  'Standard',
  'SET389',
  'SET389-KR-NP-S',
  'Set',
  'S',
  'B09KXVBD7Z',
  None,
  0,
  'INR',
  647.62,
  'MUMBAI',
  'MAHARASHTRA',
  400081.0,
  'IN',
  None,
  0,
  'Easy Ship',
  None),
 (1,
  '171-9198151-1101146',
  '04-30-22',
  'Shipped - Delivered to Buyer',
  'Merchant',
  'Amazon.in',
  'Standard',
  'JNE3781',
  'JNE3781-KR-XXXL',
  'kurta',
  '3XL',
  'B09K3WFS32',
  'Shipped',
  1,
  'INR',
  406.0,
  'BENGALURU',
  'KARNATAKA',
  560085.0,
  'IN',
  'Amazon PLCC Free-Financing Universal Merchant AAT-WNKTBO3K27EJC,Amazon PLCC Free-Financing Universal Merchant AAT-QX3UCCJESKPA2,Amazon PLCC Free-Financing Universal Merchant AAT-5QQ7BIYYQEDN2,Amazon PLCC Free-Financing Universal Merchant AAT-DSJ2QRXXWXVMQ,Amazon PLCC Free-Financing Universal Merchant AAT-CXJHMC2YJUK76,Amazon PLCC Free-Financing Universal Merchant AAT-CC4FAVTYR4X7C,Amazon PLCC Free-Financing Universal Merchant AAT-XX

In [ ]:
conn.close()

## Langchain Portion

In [ ]:
# pip install langchain langchain-experimental openai

In [ ]:
## This stuff is probably not necessary, since api key is directly specified in function later on 
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain
dotenv.load_dotenv("creds.env")

True

In [ ]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///mydatabase.db")
llm = OpenAI(openai_api_key="INSERT YOUR OPENAI API KEY HERE", temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

SELECT COUNT(*) FROM Employees


In [ ]:
## manually pulled from file, for reference
column_names = ['index',
  'Order ID',
  'Date',
  'Status',
  'Fulfilment',
  'Sales Channel ',
  'ship-service-level',
  'Style',
  'SKU',
  'Category',
  'Size',
  'ASIN',
  'Courier Status',
  'Qty',
  'currency',
  'Amount',
  'ship-city',
  'ship-state',
  'ship-postal-code',
  'ship-country',
  'promotion-ids',
  'B2B',
  'fulfilled-by',
  'Unnamed: 22']

In [ ]:
queries = "Which clothing items were ordered the most by californians" #can make more queries to see if it works well
result = db_chain.run(queries)
print(result)



> Entering new SQLDatabaseChain chain...
Which clothing items were ordered the most by californians
SQLQuery:SELECT "Style", COUNT("Style") AS "Number of Orders" FROM "Amazon Sale Report" WHERE "ship-state" = 'California' GROUP BY "Style" ORDER BY "Number of Orders" DESC LIMIT 5;
SQLResult: 
Answer:The top 5 clothing items ordered by Californians are SET389, JNE3781, JNE3371, JNE3782, and JNE3372.
> Finished chain.
The top 5 clothing items ordered by Californians are SET389, JNE3781, JNE3371, JNE3782, and JNE3372.


In [ ]:
conn.close()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cc755946-e324-4d42-82e2-8da3554143c2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>